# SETUP (restart after this)

In [ ]:
!pip install piexif

In [ ]:
# -*- coding: utf-8 -*-
# --- Setup: Google auth + Drive + NanoBanana Pro deps ---

import sys, os, subprocess, textwrap, importlib

from google.colab import auth, drive
auth.authenticate_user()
drive.mount('/content/drive')

!pip -q install --upgrade pip
!pip -q install -U "google-genai>=1.40.0" pillow numpy opencv-python-headless matplotlib gspread google-auth google-auth-oauthlib google-api-python-client piexif tqdm

print("✅ Setup done for NanoBanana Pro.")


# Select angles

In [ ]:
fr_lft = True #@param {type:"boolean"}
fr_rght = True #@param {type:"boolean"}
fr_cl = True #@param {type:"boolean"}
bc_lft = True #@param {type:"boolean"}
bc_rght = True #@param {type:"boolean"}
lft = True #@param {type:"boolean"}
rght = True #@param {type:"boolean"}
bc_ = True #@param {type:"boolean"}
fr_ = True #@param {type:"boolean"}
fr_cl_btm = False #@param {type:"boolean"}
fr_cl_tp = False #@param {type:"boolean"}


names = ["fr_lft","fr_rght","fr_cl","bc_lft","bc_rght","lft","rght","bc_","fr_","fr_cl_btm","fr_cl_tp"]
ALLOWED_BASES = [n for n in names if locals()[n]]

# CONFIG

In [ ]:
# --- Unified CONFIG ---

# Selection mode: list only
RUN_MODE = "sku_list"     #@param ["sku_list"]

# For RUN_MODE == "sku_list"
SKU_CSV = "28748, 28920, 28747, 29018, 29095, 29094"  #@param {type:"string"}

# Paths
BASE_PHOTOS_ROOT  = "/content/drive/MyDrive/Dazzl/SikSilk/SKSLK_MODELS/"
GARMENTS_ROOT     = "/content/drive/MyDrive/Dazzl/SikSilk/AlexGens/SikSilk/"


# Filename/dir policy
VALID_EXTENSIONS  = (".png", ".jpg", ".jpeg", ".PNG", ".JPG", ".JPEG")
IGNORE_DIRS       = {"old", "__MACOSX", ".ds_store", "Ricardo", "toweling"}
SKIP_FILENAME_TOKENS_CSV   = "mask, generated, freelance, _sec, _backup"   # substrings to skip
SKIP_BASENAME_SUFFIXES_CSV = "_sec"                             # stem endings to skip
REQUIRE_CUT_IN_FILENAME    = True   #@param {type:"boolean"}

# Cropping / paste-back (square 1:1, generous garment margin)
CROP_PADDING      = 100        # px padding around garment when building crop
UPPER_PADDING     = 100        # extra padding above garment
HORIZ_PADDING     = 100        # horizontal padding
MASK_EXPAND_PX    = 100        # outward growth before feather
MASK_FEATHER_PX   = 40         # Gaussian sigma for feathering
CROP_MIN_MARGIN   = 100        # minimum margin even if mask touches edge

TARGET_ASPECT = (1, 1)         # enforce square crops for 1:1 generations

# NanoBanana Pro (Google GenAI)
NANOBANANA_MODEL_ID = "gemini-3-pro-image-preview"
GEN_ASPECT_RATIO    = "1:1"
GEN_IMAGE_SIZE      = "4K"
TRYON_PROMPT = """You are an expert virtual try-on AI. You will be given a 'model image' and a 'garment image'. Your task is to create a new photorealistic image where the person from the 'model image' is wearing the clothing from the 'garment image'.

**Crucial Rules:**
1.  **Complete Garment Replacement:** You MUST completely REMOVE and REPLACE the clothing item worn by the person in the 'model image' with the new garment. No part of the original clothing (e.g., collars, sleeves, patterns) should be visible in the final image.
2.  **Preserve the Model:** The person's face, hair, body shape, and pose from the 'model image' MUST remain unchanged.
3.  **Preserve the Background:** The entire background from the 'model image' MUST be preserved perfectly.
4.  **Apply the Garment:** Realistically fit the new garment onto the person. It should adapt to their pose with natural folds, shadows, and lighting consistent with the original scene.
5.  **Output:** Return ONLY the final, edited image. Do not include any text."""

# Sheet-related (Ops removed) kept only for Gen Log appends
SPREADSHEET_ID = "1Kbq9__sEUQiuDPuza5Xy_hRyIn8pUvmfFj6vhPBrp8Y"
GEN_LOG_SHEET  = "Gen Log"

# Misc
SHOW_VISUALS = True
TIMEZONE     = "Europe/Lisbon"
OPERATOR     = "Ivan"
OUTPUT_DIR   = "/content/drive/MyDrive/Dazzl/SikSilk/SS_OUTPUT_FOLDER/19oct" #@param {type:"string"}


# === Garment/type taxonomy (kept) ===
ALLOWED_GARMENT_TYPES = [
    "hoodie","jeans","joggers","shorts","sweater","swimwear",
    "t-shirt","shirts","track top","trousers","twinset","polo","vests","shirts"
]
TOP_GARMENTS    = ["t-shirt","shirt","sweater","hoodie","track top","vest"]
BOTTOM_GARMENTS = ["shorts","jogger-trousers","trousers","jeans","swimwear"]
TWINSET_TYPES   = ["twinset"]

# === Details tokens ===
ALLOWED_DETAIL_TYPES = ["crest","logo","patch"]

# Angle sheet tokens (kept for compatibility, not used in v1.3)
ANGLE_NEEDS_REGENERATE_TOKEN = "Regenerate"
ENFORCE_BAN_SUBSTRINGS     = True
BANNED_SUBSTRINGS_CSV      = "wrong, pair, combo"
ENFORCE_REQUIRE_SUBSTRINGS = False
REQUIRED_SUBSTRINGS_CSV    = ""
REQUIRED_SUBSTRINGS_MODE   = "ANY"   # "ANY" | "ALL"

print("✅ Config ready for NanoBanana Pro v1.3")


# UTILS

In [ ]:
# --- Core utilities: normalization, angles, walking, masks (agnostic-first) ---

import os, re, fnmatch, math, uuid, pytz, random, gc, tempfile, traceback
from datetime import datetime
import numpy as np
from PIL import Image, ImageOps, ImageEnhance, ImageChops, ImageFilter

def normalize_sku_list(sku_csv: str) -> str:
    skus = []
    for raw in sku_csv.split(','):
        sku = raw.strip().upper()
        match = re.search(r'(\d+)', sku)
        if match:
            sku_number = match.group(1)
            skus.append(f"SS-{sku_number}")
    # Return as CSV string
    return ", ".join(skus)

SKU_CSV = normalize_sku_list(SKU_CSV)

# Parsers
def _parse_csv_list(s):  return [x.strip().casefold() for x in (s or "").split(",") if x.strip()]
BANNED_SUBSTRINGS       = _parse_csv_list(BANNED_SUBSTRINGS_CSV)
REQUIRED_SUBSTRINGS     = _parse_csv_list(REQUIRED_SUBSTRINGS_CSV)
SKIP_FILENAME_TOKENS    = set(_parse_csv_list(SKIP_FILENAME_TOKENS_CSV))
SKIP_BASENAME_SUFFIXES  = tuple(_parse_csv_list(SKIP_BASENAME_SUFFIXES_CSV))

# Normalizers
def _norm_sku(s):
    if s is None: return ""
    s = str(s).replace("\u00A0"," ")
    s = " ".join(s.split())
    return s.casefold()

def _norm_angle(s):
    s = (s or "").strip().lower()
    return s.strip("_ ").replace("-", "_")

# Angle aliases
ANGLE_ALIASES = {
    "fr_cl": ["fr", "fr_"],
    #"lft":   ["fr_lft", "bc_lft"],
    #"rght":  ["fr_rght", "bc_rght"],
}

# --- Helpers to keep outputs strict, sources flexible ---
def expand_as_list(angles):
    exp = list(expand_allowed_angles(angles))
    exp = [_norm_angle(a) for a in exp]
    exp.sort(key=len, reverse=True)  # prefer 'fr_cl' over 'fr'
    return exp

def pick_target_angle(source_angle: str, allowed_outputs: set) -> str | None:
    s = _norm_angle(source_angle)
    for target in allowed_outputs:
        fam = {_norm_angle(x) for x in expand_allowed_angles([target])}
        if s in fam:
            return _norm_angle(target)
    return None


def expand_allowed_angles(angles):
    expanded = set()
    for a in (angles or []):
        a_norm = _norm_angle(a)
        expanded.add(a_norm)
        for alt in ANGLE_ALIASES.get(a_norm, []):
            expanded.add(_norm_angle(alt))
    return expanded

# Ignore set
IGNORE_DIRS = {d.lower() for d in IGNORE_DIRS}

# Walkers
def _is_sku_folder(path: str) -> bool:
    if os.path.basename(os.path.normpath(path)).lower() in IGNORE_DIRS:
        return False
    try:
        for f in os.listdir(path):
            if os.path.isfile(os.path.join(path, f)) and f.lower().endswith(tuple(e.lower() for e in VALID_EXTENSIONS)):
                return True
    except Exception:
        return False
    return False

def iter_sku_folders(root: str):
    for dirpath, dirnames, filenames in os.walk(root):
        dirnames[:] = [d for d in dirnames if d.lower() not in IGNORE_DIRS]
        if any(f.lower().endswith(tuple(e.lower() for e in VALID_EXTENSIONS)) for f in filenames):
            yield dirpath

def resolve_targets(idents_csv: str, garments_root: str):
    """
    Accepts:
      • Plain SKU names, relative paths (Category/Subcategory/SKU), or absolute dirs
      • Glob patterns (e.g., 'Hoodies/*' or 'SKSLK_12*')
      • Directories that are NOT SKU leaves → expand to all descendant SKU leaves
    """
    idents = [s.strip() for s in idents_csv.replace("\n", ",").split(",") if s.strip()]
    if not idents: return [], []

    all_sku_dirs = list(iter_sku_folders(garments_root))
    rel_map = {p: os.path.relpath(p, garments_root) for p in all_sku_dirs}
    base_map = {p: os.path.basename(p) for p in all_sku_dirs}

    seen, out, unmatched = set(), [], []
    def add_path(p):
        ap = os.path.abspath(p)
        if os.path.isdir(ap):
            if _is_sku_folder(ap):
                if ap not in seen:
                    seen.add(ap); out.append(ap)
            else:
                # Expand directory to all descendant SKU leaves
                for leaf in iter_sku_folders(ap):
                    a = os.path.abspath(leaf)
                    if a not in seen:
                        seen.add(a); out.append(a)

    for ident in idents:
        before = len(out)
        # Absolute directory or SKU path
        if os.path.isabs(ident) and os.path.isdir(ident):
            add_path(ident)

        # Relative under garments root (dir or SKU)
        rel_candidate = os.path.join(garments_root, ident)
        if os.path.exists(rel_candidate):
            add_path(rel_candidate)

        # Glob/pattern over known SKU leaves (by basename or relative path)
        for p in all_sku_dirs:
            if fnmatch.fnmatch(base_map[p], ident) or fnmatch.fnmatch(rel_map[p], ident):
                add_path(p)

        if len(out) == before:
            unmatched.append(ident)

    out.sort()
    return out, unmatched

# Base/mask location resolution
def resolve_base_mask_dir(sku_folder: str,
                          garments_root: str = GARMENTS_ROOT,
                          base_root: str = BASE_PHOTOS_ROOT):
    """
    Map .../GARMENTS_ROOT/Category/Subcategory/SKU → .../BASE_ROOT/Category/Subcategory
    With robust fallbacks.
    """
    abs_sku = os.path.abspath(sku_folder)
    abs_gar = os.path.abspath(garments_root)
    try:
        rel = os.path.relpath(abs_sku, abs_gar)
    except Exception:
        rel = None

    if rel and not rel.startswith(".."):
        rel_parent = os.path.dirname(rel)
        cand = os.path.join(base_root, rel_parent)
        if os.path.isdir(cand): return cand

    subcat = os.path.basename(os.path.dirname(abs_sku))
    cat    = os.path.basename(os.path.dirname(os.path.dirname(abs_sku)))
    cand2  = os.path.join(base_root, cat, subcat)
    if os.path.isdir(cand2): return cand2

    cand3  = os.path.join(base_root, subcat)
    if os.path.isdir(cand3): return cand3
    return None

def _valid_ext(fname): return fname.lower().endswith(tuple(e.lower() for e in VALID_EXTENSIONS))

def _file_prefix_or_none(filename: str):
    low = filename.lower()
    for base in ALLOWED_BASES:
        if low.startswith(base): return base
    return None

def _find_image_with_stem_and_suffix(directory, stem, suffix=""):
    if not directory or not os.path.isdir(directory):
        return None
    stem = stem.lower()
    for file in os.listdir(directory):
        fname, fext = os.path.splitext(file)
        if fext.lower() in (".png",".jpg",".jpeg") and fname.lower() == f"{stem}{suffix}":
            return os.path.join(directory, file)
    return None

# --- Existence check in Google Drive by Colab-style path ---
def drive_file_exists_any_ext_at_colab_path(target_colab_path: str,
                                            exts=(".png", ".jpg", ".jpeg", ".PNG", ".JPG", ".JPEG")) -> bool:
    """
    Given a Colab-style *file* path (incl. a filename with any extension),
    checks if a file with the SAME stem exists in the same folder with any of the allowed extensions.
    """
    try:
        parent_id, desired_name = _resolve_parent_id_and_filename_from_colab_path(target_colab_path)
        stem, _ = os.path.splitext(desired_name)
        files = _list_children(parent_id, q_extra="")  # list once; filter locally
        allowed = {e.lower() for e in exts}
        for f in files:
            fname = f.get("name", "")
            s, e = os.path.splitext(fname)
            if s == stem and e.lower() in allowed:
                return True
        return False
    except Exception as e:
        print(f"⚠️ Ext-agnostic existence check failed for {target_colab_path}: {e}")
        return False


# === NEW: mask finding with AGNOSTIC priority ===
def find_mask_path(base_subcat_dir: str, stem_no_cut: str):
    """
    Priority:
      1) {stem}_mask_agnostic.(png|jpg|jpeg)
      2) {stem}_mask.(png|jpg|jpeg)
    """
    if not base_subcat_dir or not os.path.isdir(base_subcat_dir):
        return None

    candidates = []
    if PREFER_AGNOSTIC_MASKS:
      for ext in (".png",".jpg",".jpeg",".PNG",".JPG",".JPEG"):
          candidates.append(os.path.join(base_subcat_dir, f"{stem_no_cut}_mask_agnostic{ext}"))
    for ext in (".png",".jpg",".jpeg",".PNG",".JPG",".JPEG"):
        candidates.append(os.path.join(base_subcat_dir, f"{stem_no_cut}_mask{ext}"))

    for p in candidates:
        if os.path.exists(p):
            return p
    return None

# ──────────────────────────────────────────
# --- Aspect-ratio bbox (replaces square bbox usage) ---


def find_aspect_bbox(
    mask: Image.Image,
    aspect: tuple[int,int] = (1,1),   # width:height, e.g. (1280,1600)
    padding: int = 40,
    upper_padding: int | None = None,
    horiz_padding: int = 0,
    min_margin: int | None = None,
):
    """
    Return a rectangular bbox [x0, y0, x1, y1] that fully contains the mask + padding
    and matches the requested aspect ratio by expanding outward only.
    """
    if min_margin is None:
        try:
            min_margin = int(MASK_EXPAND_PX + 3 * MASK_FEATHER_PX + 5)
        except Exception:
            min_margin = 40

    m = np.array(mask.convert("L"))
    h, w = m.shape
    ys, xs = np.where(m > 128)
    if xs.size == 0:
        raise ValueError("Mask has no white pixels!")

    x_min, x_max = int(xs.min()), int(xs.max())
    y_min, y_max = int(ys.min()), int(ys.max())

    if upper_padding is None:
        upper_padding = padding

    # Initial padded bbox
    x0 = max(0, x_min - horiz_padding - min_margin)
    x1 = min(w, x_max + horiz_padding + min_margin)
    y0 = max(0, y_min - upper_padding - min_margin)
    y1 = min(h, y_max + padding + min_margin)

    bw, bh = (x1 - x0), (y1 - y0)
    # Desired aspect as float
    aw, ah = aspect
    target_ar = float(aw) / float(max(1, ah))

    # First pass: try to match aspect by expanding one dimension only
    def expand_to_aspect(x0, y0, x1, y1):
        bw = x1 - x0; bh = y1 - y0
        cur_ar = bw / float(max(1, bh))
        if cur_ar < target_ar:
            # too tall → need wider
            need_w = int(np.ceil(target_ar * bh))
            grow = max(0, need_w - bw)
            left_grow  = min(x0, grow // 2)
            right_grow = min(w - x1, grow - left_grow)
            x0 -= left_grow; x1 += right_grow
        elif cur_ar > target_ar:
            # too wide → need taller
            need_h = int(np.ceil(bw / target_ar))
            grow = max(0, need_h - bh)
            top_grow    = min(y0, grow // 2)
            bottom_grow = min(h - y1, grow - top_grow)
            y0 -= top_grow; y1 += bottom_grow
        return max(0,x0), max(0,y0), min(w,x1), min(h,y1)

    x0, y0, x1, y1 = expand_to_aspect(x0, y0, x1, y1)

    # Second pass: if a border capped us, re-try by expanding the other dimension
    bw, bh = (x1 - x0), (y1 - y0)
    cur_ar = bw / float(max(1, bh))
    if abs(cur_ar - target_ar) > 1e-3:
        if cur_ar < target_ar:
            # could not widen enough → try to grow height
            need_h = int(np.ceil(bw / target_ar))
            grow = max(0, need_h - bh)
            top_grow    = min(y0, grow // 2)
            bottom_grow = min(h - y1, grow - top_grow)
            y0 -= top_grow; y1 += bottom_grow
        else:
            # could not heighten enough → try to grow width
            need_w = int(np.ceil(target_ar * bh))
            grow = max(0, need_w - bw)
            left_grow  = min(x0, grow // 2)
            right_grow = min(w - x1, grow - left_grow)
            x0 -= left_grow; x1 += right_grow

    # Final clamp
    x0, y0 = max(0, int(x0)), max(0, int(y0))
    x1, y1 = min(w, int(x1)), min(h, int(y1))
    return [x0, y0, x1, y1]


# Alpha/white utilities for garment panel
WHITE_RGB = (255,255,255)
def flatten_alpha_to_white(img: Image.Image) -> Image.Image:
    if img.mode in ("RGBA","LA") or ("transparency" in img.info):
        bg = Image.new("RGB", img.size, WHITE_RGB)
        bg.paste(img, mask=img.split()[-1])
        return bg
    return img.convert("RGB")

def _tight_bbox_nonwhite_or_opaque(img: Image.Image):
    if img.mode in ("RGBA","LA") or ("transparency" in img.info):
        arr = np.asarray(img.convert("RGBA"))
        alpha = arr[...,3]
        fg = alpha > 0
    else:
        arr = np.asarray(img.convert("RGB"))
        fg = ~((arr[...,0]==255)&(arr[...,1]==255)&(arr[...,2]==255))
    if not np.any(fg): return None
    ys, xs = np.where(fg)
    x0, x1 = int(xs.min()), int(xs.max())+1
    y0, y1 = int(ys.min()), int(ys.max())+1
    return (x0,y0,x1,y1)

def crop_garment_keep_aspect(img: Image.Image) -> Image.Image:
    bbox = _tight_bbox_nonwhite_or_opaque(img)
    base = flatten_alpha_to_white(img)
    if bbox is None: return base
    full_bbox = (0,0,base.width,base.height)
    if bbox == full_bbox: return base
    return base.crop(bbox)

def to_centered_square(gar: Image.Image, fill=WHITE_RGB) -> Image.Image:
    w,h = gar.size; side = max(w,h)
    sq = Image.new("RGB", (side, side), fill)
    ox, oy = (side-w)//2, (side-h)//2
    sq.paste(gar, (ox,oy)); return sq


In [ ]:
# GPT evaluation removed for NanoBanana Pro (v1.3)


In [ ]:
# --- Visualisation helpers (restored) ---

import math
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image, ImageOps

def open_upright(path) -> Image.Image:
    # EXIF-aware loader (same as before)
    with Image.open(path) as im:
        return ImageOps.exif_transpose(im)

def show_gallery(img_list, titles=None, cols=3, w=4):
    """
    Display PIL images in a flexible grid (identical behaviour to your original).
    Only renders if SHOW_VISUALS is True.
    """
    if not globals().get("SHOW_VISUALS", False):
        return

    n = len(img_list)
    rows = math.ceil(n / cols)
    plt.figure(figsize=(cols * w, rows * w))

    for i, img in enumerate(img_list):
        plt.subplot(rows, cols, i + 1)
        # Accept PIL, torch tensors or numpy arrays (4-D batch ⇒ pick first)
        if isinstance(img, np.ndarray) and img.ndim == 4:
            img = img[0]  # (B,H,W,C) → (H,W,C)
        # Torch tensors are printed via duck-typing check to avoid hard import
        if "Tensor" in str(type(img)):
            img = img.detach().cpu().permute(1, 2, 0).numpy()
        plt.imshow(img)
        if titles and i < len(titles):
            plt.title(titles[i])
        plt.axis('off')

    plt.tight_layout()
    plt.show()


In [ ]:
# --- Paste-back (mask-aware) + crop helpers ---

import cv2

def paste_crop_back(
    full_img: Image.Image,
    edited_crop: Image.Image,
    crop_box,               # (x0,y0,x1,y1) in full_img coords
    crop_mask: np.ndarray,  # H×W uint8/bool, garment=white within crop_box
    expand_px: int = 20,    # outward dilation before feather (mask-aware)
    feather_px: int = 10,   # Gaussian σ for feathering (outside only)
    *,
    bin_thresh: int = 127,
    edge_kill_px: int | None = None,  # will be capped by actual margin
    retry_expand_px: int = 30,        # how much to enlarge bbox if needed
):
    x0, y0, x1, y1 = map(int, crop_box)
    tgt_w, tgt_h   = (x1 - x0), (y1 - y0)

    # Resize edit to crop size
    edit_rs = edited_crop.resize((tgt_w, tgt_h), Image.Resampling.LANCZOS)

    # --- 1) Binary mask in crop coordinates
    mask_np = crop_mask
    if isinstance(mask_np, Image.Image):
        mask_np = np.array(mask_np.convert("L"))
    if mask_np.ndim == 3:
        mask_np = mask_np[..., 0]
    mask_np = cv2.resize(mask_np, (tgt_w, tgt_h), interpolation=cv2.INTER_NEAREST)
    mask_bin = (mask_np > bin_thresh).astype(np.uint8)

    # --- 2) Build outside-only feather band
    if expand_px > 0:
        ksize = max(1, expand_px * 2 + 1)
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (ksize, ksize))
        dil = cv2.dilate(mask_bin, kernel, iterations=1)
    else:
        dil = mask_bin.copy()

    outside = (dil - mask_bin).clip(0, 1).astype(np.float32) * 255.0
    if feather_px > 0:
        outside = cv2.GaussianBlur(outside, (0, 0), sigmaX=feather_px, sigmaY=feather_px)

    # --- 3) Edge-kill: CAP by actual margin so we only taper near rectangle borders
    margin_left   = x0
    margin_right  = full_img.width  - x1
    margin_top    = y0
    margin_bottom = full_img.height - y1
    max_safe_edgekill = max(2, min(margin_left, margin_right, margin_top, margin_bottom))

    if edge_kill_px is None:
        edge_kill_px = int(expand_px + 3 * feather_px)
    edge_kill_px = min(int(edge_kill_px), int(max_safe_edgekill))

    # apply taper only within edge_kill band near the crop edges (no global attenuation)
    yy, xx = np.mgrid[0:tgt_h, 0:tgt_w]
    dist_edge = np.minimum.reduce([xx, tgt_w - 1 - xx, yy, tgt_h - 1 - yy]).astype(np.float32)

    edge_factor = np.ones_like(dist_edge, np.float32)
    band = dist_edge < float(edge_kill_px)
    edge_factor[band] = dist_edge[band] / float(max(1.0, edge_kill_px))
    outside *= edge_factor

    # --- 4) Final alpha: solid interior + tapered outside (no inward feathering)
    alpha = np.zeros((tgt_h, tgt_w), np.float32)
    alpha[mask_bin > 0] = 255.0
    alpha += outside
    alpha = np.clip(alpha, 0, 255).astype(np.uint8)

    # --- 5) Leakage check:
    leaks_top    = alpha[0, :].max() > 0
    leaks_bottom = alpha[-1, :].max() > 0
    leaks_left   = alpha[:, 0].max() > 0
    leaks_right  = alpha[:, -1].max() > 0

    needs_retry = False
    if leaks_top    and y0 > 0:                      needs_retry = True
    if leaks_bottom and y1 < full_img.height:        needs_retry = True
    if leaks_left   and x0 > 0:                      needs_retry = True
    if leaks_right  and x1 < full_img.width:         needs_retry = True

    if needs_retry:
        print("⚠️  alpha touches crop border (inside canvas) → retrying with expanded bbox...")
        x0n = max(0, x0 - retry_expand_px)
        y0n = max(0, y0 - retry_expand_px)
        x1n = min(full_img.width,  x1 + retry_expand_px)
        y1n = min(full_img.height, y1 + retry_expand_px)
        new_box = [x0n, y0n, x1n, y1n]

        full_mask = np.zeros((full_img.height, full_img.width), np.uint8)
        full_mask[y0:y1, x0:x1] = (mask_bin * 255).astype(np.uint8)
        region_mask_crop = full_mask[y0n:y1n, x0n:x1n]

        return paste_crop_back(
            full_img   = full_img,
            edited_crop= edited_crop,
            crop_box   = new_box,
            crop_mask  = region_mask_crop,
            expand_px  = expand_px,
            feather_px = feather_px,
            bin_thresh = bin_thresh,
            edge_kill_px = edge_kill_px + 10,
            retry_expand_px = retry_expand_px
        )

    # --- 6) Composite
    mask_img = Image.fromarray(alpha, mode="L")
    region   = full_img.crop((x0, y0, x1, y1))
    comp     = Image.composite(edit_rs, region, mask_img)
    full_img.paste(comp, (x0, y0))
    return full_img


In [ ]:

# --- NanoBanana Pro try-on (Google Cloud GenAI) ---

import os, io
from google import genai
from google.genai import types
from PIL import Image

def _load_gemini_api_key():
    key = os.getenv("GEMINI_API_KEY") or os.getenv("GEMINI_APIKEY")
    try:
        from google.colab import userdata
        key = key or userdata.get("GEMINI_API_KEY")
    except Exception:
        pass
    if not key:
        raise ValueError("Set GEMINI_API_KEY in environment or Colab userdata.")
    return key

genai_client = genai.Client(api_key=_load_gemini_api_key())

def _extract_first_image(resp):
    parts = []
    if hasattr(resp, "parts"):
        parts.extend(resp.parts)
    for cand in getattr(resp, "candidates", []):
        content = getattr(cand, "content", None)
        if content and hasattr(content, "parts"):
            parts.extend(content.parts or [])

    for part in parts:
        if isinstance(part, Image.Image):
            return part
        as_image = getattr(part, "as_image", None)
        if callable(as_image):
            img = as_image()
            if isinstance(img, Image.Image):
                return img
        inline = getattr(part, "inline_data", None)
        data = getattr(inline, "data", None) if inline else None
        if data:
            try:
                return Image.open(io.BytesIO(data))
            except Exception:
                pass
    raise ValueError("No image returned from NanoBanana Pro response.")

def run_nanobanana_tryon(model_image: Image.Image, garment_image: Image.Image,
                         *, aspect_ratio: str = GEN_ASPECT_RATIO,
                         image_size: str = GEN_IMAGE_SIZE,
                         prompt: str | None = None):
    prompt_text = prompt or TRYON_PROMPT
    resp = genai_client.models.generate_content(
        model=NANOBANANA_MODEL_ID,
        contents=[
            prompt_text,
            "Model image:",
            model_image.convert("RGB"),
            "Garment image:",
            garment_image.convert("RGB"),
        ],
        config=types.GenerateContentConfig(
            response_modalities=["IMAGE"],
            image_config=types.ImageConfig(
                aspect_ratio=aspect_ratio,
                image_size=image_size,
            ),
        ),
    )
    img = _extract_first_image(resp)
    return img.convert("RGB")

print("✅ NanoBanana Pro client ready.")


In [ ]:
# Colab cell — output routing + metadata helpers
import os, json
from PIL import PngImagePlugin

def ensure_dir(p):
    os.makedirs(p, exist_ok=True); return p

ensure_dir(OUTPUT_DIR)

def build_output_filename(sku_name: str, angle_code: str, ext=".png") -> str:
    # Examples: SS-12345-fr_rght or SS-12345-bc_lft
    angle_clean = _norm_angle(angle_code)
    return f"{sku_name}-{angle_clean}{ext}"


import json, piexif
from PIL import Image

def save_png_with_metadata(img, out_path, details_payload=None, quality=95):
    if details_payload:
        # Encode JSON as UTF-8 with an ASCII prefix per EXIF spec for UserComment
        payload = json.dumps(details_payload, ensure_ascii=False).encode("utf-8")
        user_comment = b"ASCII\x00\x00\x00" + payload  # indicates undefined/UTF-8
        exif_dict = {"0th": {}, "Exif": {piexif.ExifIFD.UserComment: user_comment}, "1st": {}, "GPS": {}, "Interop": {}}
        exif_bytes = piexif.dump(exif_dict)
        img.save(out_path, format="PNG", exif=exif_bytes)
    else:
        img.save(out_path, format="PNG")

import json, piexif
from PIL import Image, ImageOps


In [ ]:
# HYPIR removed in NanoBanana Pro v1.3


In [ ]:
# --- Google APIs: gspread + Drive upload (Operations sync removed) ---

import google.auth
SCOPES = ["https://www.googleapis.com/auth/drive", "https://www.googleapis.com/auth/spreadsheets"]
creds, _ = google.auth.default(scopes=SCOPES)

import gspread
gs = gspread.authorize(creds)

from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
drive_svc = build("drive", "v3", credentials=creds)

FOLDER_MIME   = "application/vnd.google-apps.folder"
SHORTCUT_MIME = "application/vnd.google-apps.shortcut"
PATH_PREFIX   = "/content/drive/MyDrive/"

def _escape_name(name: str) -> str: return name.replace("'", r"'")

def _maybe_follow_shortcut(file_obj):
    if file_obj.get("mimeType") == SHORTCUT_MIME:
        sd = file_obj.get("shortcutDetails", {}) or {}
        return sd.get("targetId"), sd.get("targetMimeType")
    return file_obj.get("id"), file_obj.get("mimeType")

def _list_children(parent_id: str, q_extra: str, page_size: int = 1000):
    q = f"'{parent_id}' in parents and trashed = false"
    if q_extra: q += f" and ({q_extra})"
    resp = drive_svc.files().list(
        q=q, spaces="drive", pageSize=page_size,
        fields="files(id,name,mimeType,shortcutDetails)",
        includeItemsFromAllDrives=True, supportsAllDrives=True,
    ).execute()
    return resp.get("files", [])

def _find_folder_id(parent_id: str, name: str):
    files = _list_children(
        parent_id,
        q_extra=(
            f"name = '{_escape_name(name)}' and "
            f"(mimeType = '{FOLDER_MIME}' or mimeType = '{SHORTCUT_MIME}')"
        ),
    )

    # Direct folder match
    for f in files:
        if f["mimeType"] == FOLDER_MIME:
            return f["id"]

    # Shortcut to folder
    for f in files:
        if f["mimeType"] == SHORTCUT_MIME:
            tid, tmime = _maybe_follow_shortcut(f)
            if tmime == FOLDER_MIME:
                return tid

    # Fallback: match by case-insensitive name
    files = _list_children(
        parent_id,
        q_extra=(
            f"(mimeType = '{FOLDER_MIME}' or mimeType = '{SHORTCUT_MIME}')"
        ),
    )
    needle = name.strip().casefold()

    for f in files:
        if f.get("name", "").strip().casefold() == needle:
            tid, tmime = _maybe_follow_shortcut(f)
            if tmime == FOLDER_MIME:
                return tid

    return None


def _resolve_parent_id_and_filename_from_colab_path(colab_path: str):
    if not colab_path.startswith(PATH_PREFIX):
        raise ValueError(
            f"This helper supports only '{PATH_PREFIX}...'. Got: {colab_path}"
        )

    parts = colab_path[len(PATH_PREFIX):].strip("/").split("/")
    if not parts:
        raise ValueError("Path must include a file name.")

    parent_id = "root"

    for part in parts[:-1]:
        next_id = _find_folder_id(parent_id, part)
        if not next_id:
            raise FileNotFoundError(f"Folder not found in path: '{part}'")
        parent_id = next_id

    desired_name = parts[-1]
    return parent_id, desired_name


def upload_to_drive_folder(
    local_path: str,
    parent_folder_id: str,
    desired_name: str | None = None
):
    media = MediaFileUpload(local_path, resumable=True)
    body = {
        "name": desired_name or os.path.basename(local_path),
        "parents": [parent_folder_id],
    }

    file = (
        drive_svc.files()
        .create(
            body=body,
            media_body=media,
            fields="id, webViewLink, name, parents",
            supportsAllDrives=True,
        )
        .execute()
    )

    drive_svc.permissions().create(
        fileId=file["id"],
        body={"type": "anyone", "role": "reader"},
        fields="id",
        supportsAllDrives=True,
    ).execute()

    return file


def upload_file_and_append_to_sheet(
    local_path: str,
    target_colab_path: str,
    sku_name: str,
    angle: str,
    spreadsheet_id: str | None,
    worksheet_name: str | None,
):
    parent_id, desired_name = _resolve_parent_id_and_filename_from_colab_path(
        target_colab_path
    )

    uploaded = upload_to_drive_folder(local_path, parent_id, desired_name)
    file_id = uploaded["id"]

    file_url = (
        uploaded.get("webViewLink")
        or f"https://drive.google.com/file/d/{file_id}/view?usp=sharing"
    )
    folder_url = f"https://drive.google.com/drive/folders/{parent_id}"

    if spreadsheet_id and worksheet_name:
        ts = datetime.now(pytz.timezone(TIMEZONE)).strftime("%m-%d %H:%M:%S")
        uid = str(uuid.uuid4())

        sh = gs.open_by_key(spreadsheet_id)
        ws = sh.worksheet(worksheet_name)

        sku_cell = f'=HYPERLINK("{folder_url}"; "{sku_name}")'

        ws.append_row(
            [
                sku_cell,
                angle,
                ts,
                file_url,
                uid,
                "Girls need to check",
                OPERATOR,
            ],
            value_input_option="USER_ENTERED",
        )

    return {"file_url": file_url}


#build pipe

In [ ]:
print("NanoBanana Pro client initialized above; no local diffusion pipe needed.")


# BATCH HELPERS

In [ ]:
# --- Batch processor  ---

def process_one_garment_folder(folder_path: str, allowed_angles=None):
    allowed_outputs = {_norm_angle(a) for a in (allowed_angles or [])}
    allowed_sources = expand_as_list(allowed_angles) if allowed_angles else None

    base_subcat_dir = resolve_base_mask_dir(folder_path)
    if not base_subcat_dir:
        print(f"⚠️ Cannot resolve base/mask dir for SKU: {folder_path}")
    files_sorted = sorted(os.listdir(folder_path))

    worklist = []
    for file in files_sorted:
        low = file.lower()

        if allowed_sources and not any(low.startswith(src) for src in allowed_sources):
            continue
        if SKIP_FILENAME_TOKENS and any(tok in low for tok in SKIP_FILENAME_TOKENS):
            continue
        if REQUIRE_CUT_IN_FILENAME and ("cut" not in low):
            continue
        if not _valid_ext(file):
            continue

        matched_source = next((src for src in allowed_sources if low.startswith(src)), None) if allowed_sources else None
        if not matched_source:
            continue
        target_angle = pick_target_angle(matched_source, allowed_outputs) if allowed_outputs else _norm_angle(matched_source)
        if allowed_outputs and not target_angle:
            continue

        base_img_path = _find_image_with_stem_and_suffix(base_subcat_dir, target_angle)
        if not base_img_path:
            print(f"⚠️ Missing BASE for target '{target_angle}' → skipping {file}")
            continue

        mask_path = find_mask_path(base_subcat_dir, target_angle)
        if not mask_path:
            print(f"⚠️ Missing MASK for target '{target_angle}' → skipping {file}")
            continue

        worklist.append((file, target_angle, base_img_path, mask_path))

    sku_name = os.path.basename(folder_path)
    if allowed_sources:
        print(f"▶️  {sku_name}: {len(worklist)} image(s) to generate (outputs={sorted(list(allowed_outputs))}, sources={sorted(list(set(allowed_sources)))})")
    else:
        print(f"▶️  {sku_name}: {len(worklist)} image(s) to generate")

    if not worklist:
        return

    for idx, (file, target_angle, base_img_path, mask_path) in enumerate(worklist, start=1):
        print(f"   {idx:>3}/{len(worklist):<3}  {file}  | USING STRICT base/mask='{target_angle}'")
        garment_path = os.path.join(folder_path, file)

        sku_name = os.path.basename(folder_path)
        angle_code = _norm_angle(target_angle)
        out_stem   = f"{sku_name}_{angle_code}"
        dest_check = os.path.join(OUTPUT_DIR, out_stem + ".png")

        if drive_file_exists_any_ext_at_colab_path(dest_check):
            print(f"      ⏭️  Skip: {out_stem}.(png/jpg/jpeg) already exists in {OUTPUT_DIR}")
            continue

        try:
            garment_img = flatten_alpha_to_white(open_upright(garment_path))
            base_full   = Image.open(base_img_path).convert("RGB")
            mask_full   = Image.open(mask_path).convert("L")

            show_gallery(
                [garment_img, base_full, mask_full.convert("RGB")],
                ["Source garment (white BG)", f"Base photo [{angle_code}]", f"Mask [{os.path.basename(mask_path)}]"]
            )

            bbox = find_aspect_bbox(
                mask_full,
                aspect=TARGET_ASPECT,
                padding=CROP_PADDING,
                upper_padding=UPPER_PADDING,
                horiz_padding=HORIZ_PADDING,
                min_margin=CROP_MIN_MARGIN
            )
            base_crop = base_full.crop(tuple(bbox))
            mask_crop = mask_full.crop(tuple(bbox))
            show_gallery([base_crop, mask_crop.convert("RGB"), garment_img], ["Cropped base (1:1)", "Cropped mask", "Garment (white BG)"])

            tryon_gen = run_nanobanana_tryon(
                model_image=base_crop,
                garment_image=garment_img,
                aspect_ratio=GEN_ASPECT_RATIO,
                image_size=GEN_IMAGE_SIZE,
                prompt=TRYON_PROMPT,
            )
            tryon_sq = tryon_gen.resize(base_crop.size, Image.LANCZOS)

            final_img = paste_crop_back(
                full_img   = base_full.copy(),
                edited_crop= tryon_sq,
                crop_box   = bbox,
                crop_mask  = np.array(mask_crop),
                expand_px  = MASK_EXPAND_PX,
                feather_px = MASK_FEATHER_PX
            )
            show_gallery([final_img], ["Final paste-back"])

            out_name = build_output_filename(sku_name, angle_code, ext=".png")
            tmp_path = os.path.join("/tmp", out_name)

            save_png_with_metadata(final_img, tmp_path, details_payload=None)
            target_path_for_drive = os.path.join(OUTPUT_DIR, out_name)

            info = upload_file_and_append_to_sheet(
                local_path       = tmp_path,
                target_colab_path= target_path_for_drive,
                sku_name         = sku_name,
                angle            = angle_code,
                spreadsheet_id   = SPREADSHEET_ID,
                worksheet_name   = GEN_LOG_SHEET,
            )
            print(f"      ✅ Uploaded → {info['file_url']}")

        except Exception as e:
            print(f"      ❌ Error: {e}")


In [ ]:
# --- Sheet-driven angle selection + runners ---

def build_sku_folder_index(garments_root: str):
    return { _norm_sku(os.path.basename(p)) : p for p in iter_sku_folders(garments_root) }

def run_list():
    targets, unmatched = resolve_targets(SKU_CSV, GARMENTS_ROOT)
    if not targets:
        print("⚠️ No matching SKU folders found.")
        if unmatched: print("Unmatched:", ", ".join(unmatched))
        return
    print(f"➡️  Will process {len(targets)} SKU(s).")
    for i, p in enumerate(targets, start=1):
        name = os.path.basename(p)
        print(f"\nSKU {i}/{len(targets)} ▶️  {name}")
        try:
            process_one_garment_folder(p, allowed_angles=ALLOWED_BASES)
            print(f"✅ Finished: {name}")
        except Exception as e:
            print(f"❌ Error in {name}: {e}")
    if unmatched:
        print("\nℹ️  Unmatched identifiers:")
        for u in unmatched: print("   -", u)
    print("\n🏁 List run complete.")


# DISPATCH

In [ ]:
# Dispatch
run_list()


#UNASSIGN

In [ ]:
from google.colab import runtime
runtime.unassign()